In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

pd.set_option("display.max_columns", 300)

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [14]:
# In the data directory look for data bundles (directories with 2 csv files key_events.csv and landmarks.csv)
data_dir = "data"
data_bundles = [
    os.path.join(data_dir, d)
    for d in os.listdir(data_dir)
    if os.path.isdir(os.path.join(data_dir, d))
]
print(f"Found {len(data_bundles)} data bundles:\n {"\n".join(data_bundles)}")

Found 9 data bundles:
 data\17551910629445291
data\17551914799237503
data\17551915949449192
data\17551918126689860
data\17551921956411347
data\17551929922895738
data\17551933042424453
data\17551936781740118
data\17551940576670999


In [15]:
def load_data(bundle_path):
    key_events_path = os.path.join(bundle_path, "key_events.csv")
    landmarks_path = os.path.join(bundle_path, "landmarks.csv")

    key_events = pd.read_csv(key_events_path)
    landmarks = pd.read_csv(landmarks_path)

    return key_events, landmarks


def check_left_right_mislabelling(landmarks):
    """Check if left and right hands are mislabeled by comparing wrist x-coordinates."""
    left_wrist = landmarks[(landmarks["hand_index"] == 0) & (landmarks["landmark_index"] == 0)]
    right_wrist = landmarks[(landmarks["hand_index"] == 1) & (landmarks["landmark_index"] == 0)]
    merged = pd.merge(
        left_wrist, right_wrist, on=["timestamp", "session_frame"], suffixes=("_left", "_right")
    )
    mislabeling = merged[merged["x_left"] > merged["x_right"]]
    return not mislabeling.empty


def correct_left_right_hands(landmarks):
    """Swap left and right hand labels."""
    landmarks["hand_label"] = landmarks["hand_label"].replace({"left": "tmp", "right": "left"})
    landmarks["hand_label"] = landmarks["hand_label"].replace({"tmp": "right"})

    landmarks["hand_index"] = landmarks["hand_index"].replace({0: -1, 1: 0})
    landmarks["hand_index"] = landmarks["hand_index"].replace({-1: 1})
    return landmarks


# Load all data bundles and concatenate them
all_key_events = []
all_landmarks = []
for bundle in data_bundles:
    ke, lm = load_data(bundle)

    if check_left_right_mislabelling(lm):
        print(f"Correcting left/right hand labels in bundle {bundle}")
        lm = correct_left_right_hands(lm)

    all_key_events.append(ke)
    all_landmarks.append(lm)

key_events = pd.concat(all_key_events, ignore_index=True)
landmarks = pd.concat(all_landmarks, ignore_index=True)

landmarks.drop(columns=["hand_score", "world_x", "world_y", "world_z"], inplace=True)


# landmark_index column
# 0. WRIST
# 1. THUMB_CMC
# 2. THUMB_MCP
# 3. THUMB_IP
# 4. THUMB_TIP
# 5. INDEX_FINGER_MCP
# 6. INDEX_FINGER_PIP
# 7. INDEX_FINGER_DIP
# 8. INDEX_FINGER_TIP
# 9. MIDDLE_FINGER_MCP
# 10. MIDDLE_FINGER_PIP
# 11. MIDDLE_FINGER_DIP
# 12. MIDDLE_FINGER_TIP
# 13. RING_FINGER_MCP
# 14. RING_FINGER_PIP
# 15. RING_FINGER_DIP
# 16. RING_FINGER_TIP
# 17. PINKY_MCP
# 18. PINKY_PIP
# 19. PINKY_DIP
# 20. PINKY_TIP

Correcting left/right hand labels in bundle data\17551940576670999


In [16]:
# Subtract wrist coordinates from all landmarks to make them relative to the wrist
# wrist_coords = landmarks[landmarks['landmark_index'] == 0][['timestamp', 'session_frame', 'hand_index', 'x', 'y', 'z']]
# wrist_coords.rename(columns={'x': 'wrist_x', 'y': 'wrist_y', 'z': 'wrist_z'}, inplace=True)
# landmarks = pd.merge(landmarks, wrist_coords, on=['timestamp', 'session_frame', 'hand_index'])
# landmarks['x'] = landmarks['x'] - landmarks['wrist_x']
# landmarks['y'] = landmarks['y'] - landmarks['wrist_y']
# landmarks['z'] = landmarks['z'] - landmarks['wrist_z']
# landmarks.drop(columns=['wrist_x', 'wrist_y', 'wrist_z'], inplace=True)

# Get the diffs of each joint between frames
landmarks.sort_values(by=["session_frame", "landmark_index"], inplace=True)
landmarks[["x_diff", "y_diff", "z_diff"]] = landmarks.groupby(["landmark_index", "hand_index"])[
    ["x", "y", "z"]
].diff()
landmarks.sort_values(by=["timestamp", "session_frame", "hand_index"], inplace=True)
landmarks

# # Standardize the diff columns
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# landmarks[['x', 'y', 'z']] = scaler.fit_transform(landmarks[['x', 'y', 'z']])

# diff_scaler = StandardScaler()
# landmarks[['x_diff', 'y_diff', 'z_diff']] = diff_scaler.fit_transform(landmarks[['x_diff', 'y_diff', 'z_diff']])


# # Store the transformations for testing later
# import joblib

# joblib.dump(scaler, 'scaler.pkl')
# joblib.dump(diff_scaler, 'diff_scaler.pkl')

# landmarks

,session_frame,timestamp,hand_index,hand_label,landmark_index,x,y,z,x_diff,y_diff,z_diff
0,0,5055461,0,left,0,0.228641,0.874176,1.489265e-07,NaN,NaN,NaN
1,0,5055461,0,left,1,0.310892,0.846445,4.216992e-04,NaN,NaN,NaN
2,0,5055461,0,left,2,0.376630,0.775596,-6.320109e-03,NaN,NaN,NaN
3,0,5055461,0,left,3,0.399899,0.670900,-9.071256e-03,NaN,NaN,NaN
4,0,5055461,0,left,4,0.397497,0.593852,-1.462849e-02,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2204323,8865,8350647,1,right,16,0.617561,0.435844,-1.405745e-01,0.015242,-0.025083,-0.049642
2204324,8865,8350647,1,right,17,0.769481,0.660961,-7.918941e-02,0.011564,-0.002767,-0.022972
2204325,8865,8350647,1,right,18,0.761466,0.572479,-1.117178e-01,0.016945,0.004815,-0.037114
2204326,8865,8350647,1,right,19,0.742244,0.517004,-1.316055e-01,0.021432,0.000436,-0.046470


In [17]:
%%script false --no-raise-error

# Plot the distributions of x,y,z positions of all landmarks
plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
sns.histplot(landmarks['x'], bins=50, kde=True, color='blue')
plt.title('Distribution of X Positions')
plt.xlabel('X Position')
plt.ylabel('Frequency')
plt.subplot(1, 3, 2)
sns.histplot(landmarks['y'], bins=50, kde=True, color='green')
plt.title('Distribution of Y Positions')
plt.xlabel('Y Position')
plt.ylabel('Frequency')
plt.subplot(1, 3, 3)
sns.histplot(landmarks['z'], bins=50, kde=True, color='red')
plt.title('Distribution of Z Positions')
plt.xlabel('Z Position')
plt.ylabel('Frequency')
plt.tight_layout()

# Plot the distributions of x_diff,y_diff,z_diff positions of all landmarks
plt.figure(figsize=(18, 5))
plt.subplot(2, 3, 1)
sns.histplot(landmarks['x_diff'], bins=50, kde=True, color='blue')
plt.title('Distribution of X Diff Positions')
plt.xlabel('X Diff Position')
plt.ylabel('Frequency')
plt.subplot(2, 3, 2)
sns.histplot(landmarks['y_diff'], bins=50, kde=True, color='green')
plt.title('Distribution of Y Diff Positions')
plt.xlabel('Y Diff Position')
plt.ylabel('Frequency')
plt.subplot(2, 3, 3)
sns.histplot(landmarks['z_diff'], bins=50, kde=True, color='red')
plt.title('Distribution of Z Diff Positions')
plt.xlabel('Z Diff Position')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()


Couldn't find program: 'false'


In [18]:
# Pivot the landmarks dataframe to have one row per session_frame and hand_label, and columns for each landmark's x_diff, y_diff, z_diff
landmarks_pivot = landmarks.pivot_table(
    index=["timestamp", "session_frame"],
    columns=["hand_label", "landmark_index"],
    values=["x", "y", "z", "x_diff", "y_diff", "z_diff"],
)
landmarks_pivot.columns = [f"{hand}_{idx}_{axis}" for axis, hand, idx in landmarks_pivot.columns]
landmarks_pivot.reset_index(inplace=True)
landmarks_pivot

,timestamp,session_frame,left_0_x,left_1_x,left_2_x,left_3_x,left_4_x,left_5_x,left_6_x,left_7_x,left_8_x,left_9_x,left_10_x,left_11_x,left_12_x,left_13_x,left_14_x,left_15_x,left_16_x,left_17_x,left_18_x,left_19_x,left_20_x,right_0_x,right_1_x,right_2_x,right_3_x,right_4_x,right_5_x,right_6_x,right_7_x,right_8_x,right_9_x,right_10_x,right_11_x,right_12_x,right_13_x,right_14_x,right_15_x,right_16_x,right_17_x,right_18_x,right_19_x,right_20_x,left_0_x_diff,left_1_x_diff,left_2_x_diff,left_3_x_diff,left_4_x_diff,left_5_x_diff,left_6_x_diff,left_7_x_diff,left_8_x_diff,left_9_x_diff,left_10_x_diff,left_11_x_diff,left_12_x_diff,left_13_x_diff,left_14_x_diff,left_15_x_diff,left_16_x_diff,left_17_x_diff,left_18_x_diff,left_19_x_diff,left_20_x_diff,right_0_x_diff,right_1_x_diff,right_2_x_diff,right_3_x_diff,right_4_x_diff,right_5_x_diff,right_6_x_diff,right_7_x_diff,right_8_x_diff,right_9_x_diff,right_10_x_diff,right_11_x_diff,right_12_x_diff,right_13_x_diff,right_14_x_diff,right_15_x_diff,right_16_x_diff,right_17_x_diff,right_18_x_diff,right_19_x_diff,right_20_x_diff,left_0_y,left_1_y,left_2_y,left_3_y,left_4_y,left_5_y,left_6_y,left_7_y,left_8_y,left_9_y,left_10_y,left_11_y,left_12_y,left_13_y,left_14_y,left_15_y,left_16_y,left_17_y,left_18_y,left_19_y,left_20_y,right_0_y,right_1_y,right_2_y,right_3_y,right_4_y,right_5_y,right_6_y,right_7_y,right_8_y,right_9_y,right_10_y,right_11_y,right_12_y,right_13_y,right_14_y,right_15_y,right_16_y,right_17_y,right_18_y,right_19_y,right_20_y,left_0_y_diff,left_1_y_diff,left_2_y_diff,left_3_y_diff,left_4_y_diff,left_5_y_diff,left_6_y_diff,left_7_y_diff,left_8_y_diff,left_9_y_diff,left_10_y_diff,left_11_y_diff,left_12_y_diff,left_13_y_diff,left_14_y_diff,left_15_y_diff,left_16_y_diff,left_17_y_diff,left_18_y_diff,left_19_y_diff,left_20_y_diff,right_0_y_diff,right_1_y_diff,right_2_y_diff,right_3_y_diff,right_4_y_diff,right_5_y_diff,right_6_y_diff,right_7_y_diff,right_8_y_diff,right_9_y_diff,right_10_y_diff,right_11_y_diff,right_12_y_diff,right_13_y_diff,right_14_y_diff,right_15_y_diff,right_16_y_diff,right_17_y_diff,right_18_y_diff,right_19_y_diff,right_20_y_diff,left_0_z,left_1_z,left_2_z,left_3_z,left_4_z,left_5_z,left_6_z,left_7_z,left_8_z,left_9_z,left_10_z,left_11_z,left_12_z,left_13_z,left_14_z,left_15_z,left_16_z,left_17_z,left_18_z,left_19_z,left_20_z,right_0_z,right_1_z,right_2_z,right_3_z,right_4_z,right_5_z,right_6_z,right_7_z,right_8_z,right_9_z,right_10_z,right_11_z,right_12_z,right_13_z,right_14_z,right_15_z,right_16_z,right_17_z,right_18_z,right_19_z,right_20_z,left_0_z_diff,left_1_z_diff,left_2_z_diff,left_3_z_diff,left_4_z_diff,left_5_z_diff,left_6_z_diff,left_7_z_diff,left_8_z_diff,left_9_z_diff,left_10_z_diff,left_11_z_diff,left_12_z_diff,left_13_z_diff,left_14_z_diff,left_15_z_diff,left_16_z_diff,left_17_z_diff,left_18_z_diff,left_19_z_diff,left_20_z_diff,right_0_z_diff,right_1_z_diff,right_2_z_diff,right_3_z_diff,right_4_z_diff,right_5_z_diff,right_6_z_diff,right_7_z_diff,right_8_z_diff,right_9_z_diff,right_10_z_diff,right_11_z_diff,right_12_z_diff,right_13_z_diff,right_14_z_diff,right_15_z_diff,right_16_z_diff,right_17_z_diff,right_18_z_diff,right_19_z_diff,right_20_z_diff
0,5055461,0,0.228641,0.310892,0.376630,0.399899,0.397497,0.342170,0.377155,0.393181,0.405993,0.287142,0.314736,0.332732,0.347709,0.241259,0.262496,0.283240,0.303484,0.202270,0.215876,0.229937,0.247656,0.737627,0.653781,0.598190,0.573847,0.566801,0.603901,0.536629,0.499946,0.471893,0.647479,0.588536,0.553386,0.527112,0.687138,0.646924,0.616958,0.589231,0.723499,0.725422,0.710180,0.685896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.874176,0.846445,0.775596,0.670900,0.593852,0.718563,0.573439,0.493829,0.440431,0.693069,0.527066,0.439897,0.388978,0.677768,0.527442,0.449724,0.402588,0.674438,0.549523,0.483701,0.437744,0.893587,0.867628,0.795239,0.706904,0.645536,0.747057,0.642573,0.581462,

In [19]:
from scancode_mapping import SCANCODE_MAP


delay_ms = 37

# load the key events dataset (if not already loaded)
key_events = pd.read_csv(os.path.join(data_bundles[0], "key_events.csv"))

# Create a copy of the landmarks timestamp with the delay removed.
landmarks_pivot["adjusted_timestamp"] = landmarks_pivot["timestamp"] - delay_ms

# Merge landmarks with key_events using merge_asof.
matched_frames = pd.merge_asof(
    landmarks_pivot.sort_values("adjusted_timestamp"),
    key_events.sort_values("timestamp"),
    left_on="adjusted_timestamp",
    right_on="timestamp",
    direction="nearest",
    suffixes=("", "_key"),
    tolerance=50,  # Only merge if within 50ms
)

matched_frames.sort_values(by=["timestamp", "session_frame"], inplace=True)
matched_frames["char"] = matched_frames["scancode"].map(SCANCODE_MAP).str.upper()


# Create an uppercase label consisting of the key and whether it was DOWN (pressed=1) or UP (pressed=0)
matched_frames["label"] = matched_frames.apply(
    lambda row: (
        f"{row['char']}_{'DOWN' if row['pressed'] == 1 else 'UP'}"
        if pd.notnull(row["char"])
        else "NO_KEY"
    ),
    axis=1,
)

# Drop rows where any coordinates are NaN (indicating missing hand data)
matched_frames = matched_frames.dropna(
    subset=[
        col for col in matched_frames.columns if any(axis in col for axis in ["_x", "_y", "_z"])
    ]
)
matched_frames

,timestamp,session_frame,left_0_x,left_1_x,left_2_x,left_3_x,left_4_x,left_5_x,left_6_x,left_7_x,left_8_x,left_9_x,left_10_x,left_11_x,left_12_x,left_13_x,left_14_x,left_15_x,left_16_x,left_17_x,left_18_x,left_19_x,left_20_x,right_0_x,right_1_x,right_2_x,right_3_x,right_4_x,right_5_x,right_6_x,right_7_x,right_8_x,right_9_x,right_10_x,right_11_x,right_12_x,right_13_x,right_14_x,right_15_x,right_16_x,right_17_x,right_18_x,right_19_x,right_20_x,left_0_x_diff,left_1_x_diff,left_2_x_diff,left_3_x_diff,left_4_x_diff,left_5_x_diff,left_6_x_diff,left_7_x_diff,left_8_x_diff,left_9_x_diff,left_10_x_diff,left_11_x_diff,left_12_x_diff,left_13_x_diff,left_14_x_diff,left_15_x_diff,left_16_x_diff,left_17_x_diff,left_18_x_diff,left_19_x_diff,left_20_x_diff,right_0_x_diff,right_1_x_diff,right_2_x_diff,right_3_x_diff,right_4_x_diff,right_5_x_diff,right_6_x_diff,right_7_x_diff,right_8_x_diff,right_9_x_diff,right_10_x_diff,right_11_x_diff,right_12_x_diff,right_13_x_diff,right_14_x_diff,right_15_x_diff,right_16_x_diff,right_17_x_diff,right_18_x_diff,right_19_x_diff,right_20_x_diff,left_0_y,left_1_y,left_2_y,left_3_y,left_4_y,left_5_y,left_6_y,left_7_y,left_8_y,left_9_y,left_10_y,left_11_y,left_12_y,left_13_y,left_14_y,left_15_y,left_16_y,left_17_y,left_18_y,left_19_y,left_20_y,right_0_y,right_1_y,right_2_y,right_3_y,right_4_y,right_5_y,right_6_y,right_7_y,right_8_y,right_9_y,right_10_y,right_11_y,right_12_y,right_13_y,right_14_y,right_15_y,right_16_y,right_17_y,right_18_y,right_19_y,right_20_y,left_0_y_diff,left_1_y_diff,left_2_y_diff,left_3_y_diff,left_4_y_diff,left_5_y_diff,left_6_y_diff,left_7_y_diff,left_8_y_diff,left_9_y_diff,left_10_y_diff,left_11_y_diff,left_12_y_diff,left_13_y_diff,left_14_y_diff,left_15_y_diff,left_16_y_diff,left_17_y_diff,left_18_y_diff,left_19_y_diff,left_20_y_diff,right_0_y_diff,right_1_y_diff,right_2_y_diff,right_3_y_diff,right_4_y_diff,right_5_y_diff,right_6_y_diff,right_7_y_diff,right_8_y_diff,right_9_y_diff,right_10_y_diff,right_11_y_diff,right_12_y_diff,right_13_y_diff,right_14_y_diff,right_15_y_diff,right_16_y_diff,right_17_y_diff,right_18_y_diff,right_19_y_diff,right_20_y_diff,left_0_z,left_1_z,left_2_z,left_3_z,left_4_z,left_5_z,left_6_z,left_7_z,left_8_z,left_9_z,left_10_z,left_11_z,left_12_z,left_13_z,left_14_z,left_15_z,left_16_z,left_17_z,left_18_z,left_19_z,left_20_z,right_0_z,right_1_z,right_2_z,right_3_z,right_4_z,right_5_z,right_6_z,right_7_z,right_8_z,right_9_z,right_10_z,right_11_z,right_12_z,right_13_z,right_14_z,right_15_z,right_16_z,right_17_z,right_18_z,right_19_z,right_20_z,left_0_z_diff,left_1_z_diff,left_2_z_diff,left_3_z_diff,left_4_z_diff,left_5_z_diff,left_6_z_diff,left_7_z_diff,left_8_z_diff,left_9_z_diff,left_10_z_diff,left_11_z_diff,left_12_z_diff,left_13_z_diff,left_14_z_diff,left_15_z_diff,left_16_z_diff,left_17_z_diff,left_18_z_diff,left_19_z_diff,left_20_z_diff,right_0_z_diff,right_1_z_diff,right_2_z_diff,right_3_z_diff,right_4_z_diff,right_5_z_diff,right_6_z_diff,right_7_z_diff,right_8_z_diff,right_9_z_diff,right_10_z_diff,right_11_z_diff,right_12_z_diff,right_13_z_diff,right_14_z_diff,right_15_z_diff,right_16_z_diff,right_17_z_diff,right_18_z_diff,right_19_z_diff,right_20_z_diff,adjusted_timestamp,timestamp_key,vkey,scancode,pressed,char,label
1,5055492,1,0.234707,0.317121,0.375668,0.398289,0.399404,0.340957,0.374374,0.391504,0.404666,0.287503,0.311305,0.330960,0.349131,0.242625,0.258530,0.282789,0.307750,0.206063,0.213423,0.229433,0.251045,0.733924,0.651999,0.595314,0.569354,0.565623,0.601481,0.529485,0.496363,0.473402,0.647138,0.583467,0.547322,0.519156,0.690159,0.642938,0.611613,0.583554,0.729328,0.723667,0.706809,0.684053,-0.033385,-0.033451,-0.039764,-0.045675,-0.041637,-0.029272,-0.026919,-0.024020,-0.021438,-0.027088,-0.030846,-0.030715,-0.028856,-0.026235,-0.029104,-0.025667,-0.021695,-0.025162,-0.025076,-0.024177,-0.021843,-0.017765,-0.022139,-0.016735,-0.004883,0.007629,-0.049488,-0.068622,-0.073398,-0.074506,-0.050551,-0.063209,-0.065785,-0.067798,-0.046920,-0.060791,-0.0

In [20]:
# Remove unnecessary columns
data = matched_frames.drop(
    columns=["timestamp_key", "adjusted_timestamp", "vkey", "scancode", "char", "pressed"]
)
# Remove wrist columns
data = data.drop(columns=[col for col in data.columns if "_0" in col])

The input is of size 4x10x6, structured in the following format:
```plaintext
(pinky)      (ring)       (middle)    (index)      (thumb)     (thumb)      (index)      (middle)      (ring)        (pinky)       
left_20_t    left_16_t    left_12_t    left_8_t    left_4_t    right_4_t    right_8_t    right_12_t    right_16_t    right_20_t                
left_19_t    left_15_t    left_11_t    left_7_t    left_3_t    right_3_t    right_7_t    right_11_t    right_15_t    right_19_t           
left_18_t    left_14_t    left_10_t    left_6_t    left_2_t    right_2_t    right_6_t    right_10_t    right_14_t    right_18_t           
left_17_t    left_13_t    left_9 _t    left_5_t    left_1_t    right_1_t    right_5_t    right_9 _t    right_13_t    right_17_t      
```
where `t` is `_x | _y | _z | _x_diff | _y_diff | _z_diff` (6 channels) 

In [21]:
# Create one input sample of size 4x10x6 for each row in data
joint_cols = [
    col
    for col in data.columns
    if any(axis in col for axis in ["_x", "_y", "_z", "_x_diff", "_y_diff", "_z_diff"])
]

# Use to_numpy() for efficiency
input_values = data[joint_cols].to_numpy(dtype=float)

# Reshape the entire array at once
input_values = input_values.reshape(-1, 6, 10, 4)

# Concatenate along the second axis
input_values = np.concatenate(
    [input_values[:, :, :5, :][:, :, ::-1, :], input_values[:, :, 5:, :]], axis=2
)

# Assign the NumPy array to the 'data' column
data = data.copy()
data.drop(columns=joint_cols, inplace=True)
data.rename(columns={"session_frame": "frame_index", "label": "y"}, inplace=True)
data["X"] = [row.tolist() for row in input_values]

In [33]:
import numpy as np
from numba import jit

@jit(nopython=True)
def fast_clipped_cumsum(data: np.ndarray) -> np.ndarray:
  """
  A fast, Numba-compiled version for NumPy arrays.
  """
  # Pre-allocate a NumPy array for the results for efficiency
  results = np.zeros_like(data)

  for i in range(data.shape[0]):
        if i == 0:
            results[i] = data[i]
        results[i] = (results[i - 1] + data[i]).clip(0,1)

  return results

KEY_POSITION_MAPPING = {
    "ESC": (0,0), "F1": (0,1), "F2": (0,2), "F3": (0,3), "F4": (0,4), "F5": (0,5), "F6": (0,6), "F7": (0,7), "F8": (0,8), "F9": (0,9), "F10": (0,10), "F11": (0,11), "F12": (0,12), "INS": (0,13), "DEL": (0,14),
    "`": (1,0), "1": (1,1), "2": (1,2), "3": (1,3), "4": (1,4), "5": (1,5), "6": (1,6), "7": (1,7), "8": (1,8), "9": (1,9), "0": (1,10), "-": (1,11), "=": (1,12), "BKSP": [(1,13), (1,14)],
    "TAB": (2,0), "Q": (2,1), "W": (2,2), "E": (2,3), "R": (2,4), "T": (2,5), "Y": (2,6), "U": (2,7), "I": (2,8), "O": (2,9), "P": (2,10), "[": (2,11), "]": (2,12), "ENTR": [(2,13), (2,14), (3,13), (3,14)],
    "CAPS": (3,0), "A": (3,1), "S": (3,2), "D": (3,3), "F": (3,4), "G": (3,5), "H": (3,6), "J": (3,7), "K": (3,8), "L": (3,9), ";": (3,10), "'": (3,11), "\\": [(3,12), (4,1)],
    "LSHFT": (4,0), "Z": (4,2), "X": (4,3), "C": (4,4), "V": (4,5), "B": (4,6), "N": (4,7), "M": (4,8), ",": (4,9), ".": (4,10), "/": (4,11), "RSHFT": [(4,12), (4,13), (4,14)],
    "LCTRL": (5,0), "FN": (5,1), "META": (5,2), "LALT": (5,3), "SPC": [(5,4), (5,5), (5,6), (5,7), (5,8)], "RALT": (5,9), "RCTRL": (5,10), "LEFT": (5,11), "UP": (5,12), "DOWN": (5,13), "RIGHT": (5,14)
}

# Map the labels to a one-hot matrix based on the KEY_POSITION_MAPPING
def label_to_one_hot(label):
    key, action = label.split("_")
    one_hot = np.zeros((6, 15), dtype=int)
    if label == "NO_KEY":
        return one_hot
    positions = KEY_POSITION_MAPPING.get(key, [])
    if not isinstance(positions, list):
        positions = [positions]
    for pos in positions:
        if action == "DOWN":
            one_hot[pos] = 1
        else:
            one_hot[pos] = -1
    return one_hot

input_data = data.copy()
input_data["y"] = input_data["y"].apply(label_to_one_hot)
cum_one_hot = fast_clipped_cumsum(np.stack(input_data["y"].to_numpy()))
input_data["key_state"] = list(cum_one_hot)

# input_data["key_state"] = input_data["y"].cumsum()
# # Reset the index so that indexing in the loop works correctly.
# input_data = input_data.reset_index(drop=True)
# input_data.to_hdf(f"processed_data_{input_data['timestamp'].min()}_{input_data['timestamp'].max()}.h5", key='df', index=False)